In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from tpot import TPOTRegressor
from sklearn.metrics import make_scorer
import xgboost as xg 
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data = pd.read_csv('merged_stat_score_data_clean.csv')
player_stats = pd.read_csv("player_stats.csv")
# player_stats.columns()
player_stats['home.team.name'].unique()
player_stats['season'] = [i.strftime('%Y') for i in pd.to_datetime(player_stats['utcStartTime']).to_list()]

C:\Users\whitehes\AppData\Local\Temp\ipykernel_22108\1297226192.py:2: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  player_stats = pd.read_csv("player_stats.csv")


In [83]:
features = ['kicks', 'handballs', 'disposals', 'marks', 'bounces', 'tackles',
       'contestedPossessions', 'uncontestedPossessions', 'totalPossessions',
       'inside50s', 'marksInside50', 'contestedMarks', 'hitouts',
       'onePercenters', 'disposalEfficiency', 'clangers', 'freesFor',
       'freesAgainst', 'rebound50s', 'goalAssists', 'turnovers', 'intercepts',
       'tacklesInside50', 'shotsAtGoal', 'scoreInvolvements', 'metresGained',
       'clearances.centreClearances', 'clearances.stoppageClearances',
       'clearances.totalClearances', 'extendedStats.effectiveKicks',
       'extendedStats.kickToHandballRatio', 'extendedStats.effectiveDisposals',
       'extendedStats.marksOnLead', 'extendedStats.interceptMarks',
       'extendedStats.hitoutsToAdvantage', 'extendedStats.groundBallGets',
       'extendedStats.f50GroundBallGets', 'extendedStats.scoreLaunches',
       'extendedStats.pressureActs', 'extendedStats.defHalfPressureActs',
       'extendedStats.spoils', 'extendedStats.ruckContests',
       'extendedStats.contestDefOneOnOnes', 'extendedStats.contestDefLosses',
       'extendedStats.contestOffOneOnOnes', 'extendedStats.contestOffWins',
       'extendedStats.centreBounceAttendances', 'extendedStats.kickins',
       'extendedStats.kickinsPlayon']
target = ['score']

In [61]:
def generate_test_train_split(data, features, target, test_size=0.4):
    X = data[features]
    y = data[target]

    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
    
    return {"x_train":x_train,"x_test":x_test,"y_train":y_train,"y_test":y_test}

In [95]:
def tpot_pipeline(data, features, target, size=20, scoring='r2'):
    data_ = generate_test_train_split(data, features, target)

    my_scorer = make_scorer(mean_squared_error, greater_is_better=False)

    pipeline_optimizer = TPOTRegressor(generations=5, population_size=size, verbosity=2, scoring=scoring)

    pipeline_optimizer.fit(data_['x_train'], list(data_['y_train'].score))
    
    print(pipeline_optimizer.score(data_['x_test'], data_['y_test']))
    
    return pipeline_optimizer

In [96]:
tpot_pipeline(data, features, target)

Optimization Progress:   0%|          | 0/120 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.9024088695485031

Generation 2 - Current best internal CV score: 0.9024088695485031

Generation 3 - Current best internal CV score: 0.9035413671932021

Generation 4 - Current best internal CV score: 0.9035413671932021

Generation 5 - Current best internal CV score: 0.9035413671932021

Best pipeline: RidgeCV(ElasticNetCV(ZeroCount(input_matrix), l1_ratio=0.65, tol=1e-05))
0.9153528457736189


C:\Users\whitehes\Anaconda3\lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


TPOTRegressor(generations=5, population_size=20, scoring='r2', verbosity=2)

In [298]:
def fit_lin_regression(x_train, y_train):

    # fit linear regression line
    regr_model = LinearRegression()
    regr_model.fit(x_train, y_train)
    
    return regr_model

In [299]:
def eval_lin_regression(x_test, y_test):
    # predict the values
    pred = regr.predict(x_test)
    
    # Regression Score 
    print("Regression Score (R-square): " + str(regr.score(x_test, y_test)))

    #RMSE
    print("RMSE: " + str(mean_squared_error(y_test, pred)**0.5))

In [301]:
def fit_xgboost(x_train, y_train):# Instantiation 
    xgb_r = xg.XGBRegressor(objective ='reg:squarederror', 
                      n_estimators = 10, seed = 42) 

    # Fitting the model 
    xgb_r.fit(x_train, y_train)
    
    return xgb_r

In [300]:
def eval_xgboost(x_test, y_test):
    # Predict the model 
    pred = xgb_r.predict(x_test) 

    # RMSE Computation 
    rmse = np.sqrt(mean_squared_error(y_test, pred)) 
    print("RMSE : % f" %(rmse)) 

In [9]:
def return_stats_input(data, home_team, away_team, model_inputs, games=5):
    '''Take in player_stats and returns aggregated team stats for previous home/away team matchups'''
    
    if 'team.name' not in model_inputs:
        model_inputs.append('team.name')
        
    match_ups = data[(data['home.team.name']==home_team) & (data['away.team.name']==away_team)]
    group_by = ['season','round.roundNumber','team.name']
    team_stats = match_ups.groupby(group_by).sum()
    team_stats.reset_index(inplace=True)
    
    home_stats = team_stats[team_stats['team.name']==home_team][model_inputs].groupby('team.name').mean()
    
    away_stats = team_stats[team_stats['team.name']==away_team][model_inputs].groupby('team.name').mean()
    
    return (home_stats, away_stats)

In [148]:
player_stats['home.team.name'].unique()

array(['Carlton', 'Collingwood', 'Melbourne', 'Adelaide Crows',
       'Western Bulldogs', 'Brisbane Lions', 'St Kilda', 'GWS Giants',
       'Fremantle', 'Richmond', 'Sydney Swans', 'Essendon',
       'Port Adelaide', 'Geelong Cats', 'West Coast Eagles',
       'North Melbourne', 'Hawthorn', 'Gold Coast Suns'], dtype=object)

In [10]:
round_matchups = [
    ('Sydney Swans','Melbourne'),
    ('Brisbane Lions','Carlton'),
    ('Gold Coast Suns','Richmond'),
    ('GWS Giants','Collingwood')]

In [15]:
def generate_tear_off(model, round_matchups, player_stats,model_inputs):
    print('-------------------LINEAR REGRESSION TEAR-OFF-------------------')
    for tup in round_matchups:
        print("Home: " + tup[0] + '\t' + "Away: " + tup[1])
        print(f"{model.predict(return_stats_input(player_stats,tup[0],tup[1],model_inputs)[0])}" + '\t\t\t' + \
             f"{model.predict(return_stats_input(player_stats,tup[0],tup[1],model_inputs)[1])}")

In [70]:
def generate_tear_off(model, round_matchups, player_stats,model_inputs):
    print('-------------------' + str(pipeline_optimizer.fitted_pipeline_[0]).replace('()','') + ' TEAR-OFF-------------------')
    for tup in round_matchups:
        print("Home: " + tup[0] + '\t' + "Away: " + tup[1])
        print(f"{round(model.predict(return_stats_input(player_stats,tup[0],tup[1],model_inputs)[0])[0])}" + '\t\t\t' + \
             f"{round(model.predict(return_stats_input(player_stats,tup[0],tup[1],model_inputs)[1])[0])}")

In [71]:
generate_tear_off(pipeline_optimizer, round_matchups, player_stats, features)

-------------------RobustScaler TEAR-OFF-------------------
Home: Sydney Swans	Away: Melbourne
73			77
Home: Brisbane Lions	Away: Carlton
96			69
Home: Gold Coast Suns	Away: Richmond
72			101
Home: GWS Giants	Away: Collingwood
88			74


In [75]:
round1 = [('Carlton', 'Richmond'),
('Collingwood','Sydney Swans'),
('Essendon','Hawthorn'),
('GWS Giants','North Melbourne'),
('Geelong Cats','St Kilda'),
('Gold Coast Suns','Adelaide Crows'),
('Melbourne','Western Bulldogs'),
('Port Adelaide','West Coast Eagles'),
('Fremantle','Brisbane Lions')]

In [76]:
generate_tear_off(pipeline_optimizer, round1, player_stats, features)

-------------------RobustScaler TEAR-OFF-------------------
Home: Carlton	Away: Richmond
86			85
Home: Collingwood	Away: Sydney Swans
59			48
Home: Essendon	Away: Hawthorn
96			88
Home: GWS Giants	Away: North Melbourne
59			76
Home: Geelong Cats	Away: St Kilda
95			60
Home: Gold Coast Suns	Away: Adelaide Crows
84			87
Home: Melbourne	Away: Western Bulldogs
103			68
Home: Port Adelaide	Away: West Coast Eagles
100			45
Home: Fremantle	Away: Brisbane Lions
75			88
